In [3]:
import pandas as pd
import numpy as np
from pathlib import Path

# 2025-08-30

REPO_ROOT = Path.cwd().parent
csv_path = REPO_ROOT / "data" / "driving_log.csv"

# 디버그 출력 (경로 이슈 확인용)
print("CWD:", Path.cwd())
print("CSV PATH:", csv_path)
print("EXISTS:", csv_path.exists())

# 2025-08-30
# 파일이 없을 때는 조용히 넘어가지 말고 바로 실패시키는 게 안전하다.
# (검증/재현 관점에서 "다른 파일을 읽어버리는 실수"가 더 위험)
if not csv_path.exists():
    raise FileNotFoundError(
        "driving_log.csv not found. Place it under ./data/ (repo root 기준) "
        "and run this notebook from the driving-log-analysis/notebook directory "
        "or ensure the repo structure is intact."
    )

df = pd.read_csv(csv_path)

# ==============================
# 2. 임계값 설정 (m/s^2)
# ==============================
HARD_ACCEL_THRESHOLD = 3.0
HARD_BRAKE_THRESHOLD = -3.0

# ==============================
# 3. 급가속 / 급제동 이벤트 탐지
# ==============================
hard_accel_df = df[df["accel_mps2"] > HARD_ACCEL_THRESHOLD]
hard_brake_df = df[df["accel_mps2"] < HARD_BRAKE_THRESHOLD]

hard_accel_count = len(hard_accel_df)
hard_brake_count = len(hard_brake_df)
total_hard_events = hard_accel_count + hard_brake_count

# ==============================
# 4. 주행 시간 / 거리 계산
# ==============================
total_time_sec = df["time_sec"].max() - df["time_sec"].min()

avg_speed_kmh = df["speed_kmh"].mean()
distance_km = avg_speed_kmh * (total_time_sec / 3600) if total_time_sec > 0 else 0

# ==============================
# 5. 이벤트 빈도 계산 (안정성 처리)
# ==============================
MIN_TIME_SEC = 60      # 최소 1분 이상 데이터
MIN_DIST_KM = 0.5      # 최소 0.5km 이상 주행

events_per_min = (total_hard_events / (total_time_sec / 60)) if total_time_sec >= MIN_TIME_SEC else None
events_per_km = (total_hard_events / distance_km) if distance_km >= MIN_DIST_KM else None

# ==============================
# 6. 분류 함수
# ==============================
# 이벤트 총량 기준 분류 (짧은 데이터에도 동작하지만 과대평가 위험 존재)
def classify_by_count(total_events):
    if total_events <= 2:
        return "Smooth Driver"
    elif total_events <= 5:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

# 시간 정규화 기준 분류 (데이터가 짧으면 Insufficient Data로 보수 처리)
def classify_by_rate(events_per_min):
    if events_per_min is None:
        return "Insufficient Data"
    elif events_per_min < 3:
        return "Smooth Driver"
    elif events_per_min < 6:
        return "Normal Driver"
    else:
        return "Aggressive Driver"

# ==============================
# 7. 분류 결과
# ==============================
driver_type_count = classify_by_count(total_hard_events)
driver_type_rate = classify_by_rate(events_per_min)

# ==============================
# 8. 최종 출력
# ==============================
print("\nDriver Behavior Analysis Result")
print("-" * 40)
print(f"Data Path               : {csv_path}")
print(f"Hard Acceleration Events : {hard_accel_count}")
print(f"Hard Braking Events      : {hard_brake_count}")
print(f"Total Hard Events        : {total_hard_events}")
print(f"Total Time (sec)         : {total_time_sec:.1f}")
print(f"Distance (km)            : {distance_km:.3f}")

print("-" * 40)
print(f"Events per Minute        : {events_per_min:.2f}" if events_per_min is not None else "Events per Minute        : Insufficient Data")
print(f"Events per km            : {events_per_km:.2f}" if events_per_km is not None else "Events per km            : Insufficient Data")

print("-" * 40)
print(f"Classification (Count)   : {driver_type_count}")
print(f"Classification (Rate)    : {driver_type_rate}")


CWD: C:\Users\VIEW LIFE\driving-log-analysis\notebook
CSV PATH: C:\Users\VIEW LIFE\driving-log-analysis\data\driving_log.csv
EXISTS: True

Driver Behavior Analysis Result
----------------------------------------
Data Path               : C:\Users\VIEW LIFE\driving-log-analysis\data\driving_log.csv
Hard Acceleration Events : 3
Hard Braking Events      : 3
Total Hard Events        : 6
Total Time (sec)         : 9.0
Distance (km)            : 0.064
----------------------------------------
Events per Minute        : Insufficient Data
Events per km            : Insufficient Data
----------------------------------------
Classification (Count)   : Aggressive Driver
Classification (Rate)    : Insufficient Data
